C5-NC5 indicates that the MI-score with the cut-off point three was used as the association measure; the collocates were identified in the span of three words to the left (L3) and three words to the right (R3) of the node and the frequency threshold was five for both the collocate (C5) and the collocation (NC5). (frequency threshold exists)

Generally, the smaller the span, the greater the focus of the analysis on the most immediate lexico-grammatical patterns; a larger span captures looser associations.

TO DO
- make the freq threshold vary based on the amount of data for each year gap!!! (done, but might need adjusting)
- rerun with 3l3r and 5l5r for comparison (done)
- avoid lemmatization but tokenize (fact/facts) (done)
- include as a side if its from rstl rstb or rsta (done)
- different freq thresholds for fact/facts since they are not as common --> calculating the minimum subcorpora amount for the frequency (done, technically)

minimum subcorpora:
for every 412k, there should be 3

Period          Total Words  Fact     Facts   
1715-1724       412012       3        3


however, the scaling should be different since theres less of facts overall? though they align on the smallest subcorpus

current dealing of old language makes "nasal" "coelimic" "secretory" significant when they shouldnt be (its likely "tracts")

theres some bad redundancy in the code, its likely largely unoptimized and not as concise as it could be

In [ ]:
import nltk
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.tokenize import word_tokenize
import string
import os
import re
from collections import defaultdict
from tqdm.notebook import tqdm
import traceback
import concurrent.futures

nltk.download('punkt')

def extract_year_from_filename(filename):
    match = re.search(r'rst[bla]?_(\d{4})', filename) #doesnt detect some of the late files (past 2000s) but that doesnt matter for now
    return int(match.group(1)) if match else None

#maybe make it only replace those which stand alone as words, not in the middle of a word
def clean_text(text):
    replacements = {
        'ſact': 'fact',
        #'ſacſ': 'facts', #these are too rare to be worth it and can be confused with "tracts"
        #'ſacts': 'facts',
        #'ſacſs': 'facts'  
    }
    for old_pattern, replacement in replacements.items():
        text = re.sub(old_pattern, replacement, text, flags=re.IGNORECASE)
    #text = text.replace('ſ', 's').replace('Å¿', 's') #chose not to do this, since ſ is more complicated than that
    return text

def build_file_directory_map(text_dirs): #to get the directory and filename later, hopefully the maps match correctly
    file_directory_map = {}
    for dir_path in text_dirs:
        dir_name = os.path.basename(dir_path)
        for filename in os.listdir(dir_path):
            if filename.endswith('.txt'):
                file_path = os.path.join(dir_path, filename)
                try:
                    with open(file_path, 'r', encoding='utf-8') as file:
                        text = file.read()
                        cleaned_text = clean_text(text)
                        file_directory_map[cleaned_text] = (dir_name, filename)
                except Exception as e:
                    print(f"Error reading file {file_path}: {str(e)}")
    return file_directory_map

def process_file(file_path):
    try:
        year = extract_year_from_filename(os.path.basename(file_path))
        if not year:
            return None
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
        cleaned_text = clean_text(text) #make sure the text is cleaned!
        return year, cleaned_text
    except Exception as e:
        print(f"Error processing file {file_path}: {str(e)}")
        return None

def process_tokens(text):
    clean_text(text) #making sure the text is cleaned, likely redundant
    tokens = word_tokenize(text.lower())
    return [token for token in tokens if token.isalpha() and token not in string.punctuation] #omitting punctuation and non-alphabetic characters

def calculate_scaled_freq_filter(total_words, word_of_interest):
    if word_of_interest == "fact" or "facts":
        base_threshold = 3  # base threshold for the lowest amount of words (412,012) using smallest subcorpora for "fact"
        scale_factor = 3 / 412012  #increase threshold by this factor for each extra word over 412012
        if total_words <= 412012:
            return base_threshold
        else:
            additional_threshold = (total_words - 412012) * scale_factor
            return base_threshold + round(additional_threshold) #needs to be rounded as integer
    
    #alternative differing threshold that i tried and in the end rejected (didn't make a meaningful enough difference and may have been too lax, that said, hard to say)
    #elif word_of_interest == "facts": 
        #base_threshold = 2 #theres much less of facts than fact (sometimes 0)
        #scale_factor = 2 / 411208 #should've maybe used 2, as the smallest subcorp has 2 facts
        #if total_words <= 411208:
        #    return base_threshold
        #else:
        #   additional_threshold = (total_words - 411208) * scale_factor
        #   return base_threshold + round(additional_threshold)


def process_window(window_data, word_of_interest, collocate_window, file_directory_map):
    try:
        window_start, window_end, window_texts = window_data
        all_tokens = [token for text in window_texts for token in process_tokens(text)]
        
        total_tokens = len(all_tokens)
        total_texts = len(window_texts)
        
        freq_filter = calculate_scaled_freq_filter(total_tokens, word_of_interest)
        print(f"Window {window_start}-{window_end}: Using frequency filter of {freq_filter} for {total_tokens} total words")
        
        finder = BigramCollocationFinder.from_words(all_tokens, window_size=collocate_window)
        finder.apply_freq_filter(freq_filter)
        
        collocations = finder.score_ngrams(BigramAssocMeasures().pmi)
        
        collocate_stats = []
        collocate_contexts = defaultdict(list)
        collocate_docs = defaultdict(set)  #which document collocates appear in

        word_of_interest_lower = word_of_interest.lower()
        word_of_interest_freq = sum(finder.word_fd[word] for word in finder.word_fd if word.lower() == word_of_interest_lower)

        text_sources = {text: file_directory_map.get(text, ("Unknown", "Unknown")) for text in window_texts}

        for bigram, pmi in collocations:
         if word_of_interest_lower in (word.lower() for word in bigram):
            other_word = bigram[0] if bigram[1].lower() == word_of_interest_lower else bigram[1]
            
            #track the filename of where the collocation occurs (so can only keep collocations that appear in more than one text)
            for text in window_texts:
                    dir_name, filename = text_sources[text]
                    collocate_docs[(word_of_interest_lower, other_word.lower())].add(filename)  # Store the filename for this collocation

            #other statistics
            observed_freq = finder.ngram_fd[bigram]
            word_freq = finder.word_fd[other_word]
            expected_freq = (word_of_interest_freq * word_freq) / total_tokens
            num_docs = len(collocate_docs[(word_of_interest_lower, other_word.lower())])
            #print(f"Window {window_start}-{window_end}: Collocate ('{word_of_interest_lower}', '{other_word.lower()}') found in {num_docs} documents.") #rn its looking only at the texts in the window, not the whole corpus
            
        # Only keep collocates that appear in more than one text (collocation must appear in more than one document)
            if num_docs > 1:
                collocate_stats.append({
                    'word': other_word,
                    'total_corpus': word_freq,
                    'expected_freq': expected_freq,
                    'observed_freq': observed_freq,
                    'num_texts': num_docs,  
                    'pmi': pmi
                })

            for text in window_texts:
                        words = text.split()
                        for i, word in enumerate(words):
                            if word.lower() == word_of_interest_lower and other_word.lower() in [w.lower() for w in words[max(0, i-collocate_window):i+collocate_window+1]]:
                                context = ' '.join(words[max(0, i-collocate_window):i+collocate_window+1])
                                dir_name, filename = text_sources[text]
                                context_info = context
                                collocate_contexts[other_word].append((context_info, other_word, filename, dir_name, freq_filter))
        
        collocate_stats.sort(key=lambda x: x['pmi'], reverse=True)
        filtered_contexts = [context for collocate in collocate_stats for context in collocate_contexts.get(collocate['word'], [])]

        return f"{window_start}-{window_end}", {
            'collocates': collocate_stats,
            'total_tokens': total_tokens,
            'total_texts': total_texts,
            'contexts': filtered_contexts
        }
    except Exception as e:
        print(f"Error processing window {window_start}-{window_end}: {str(e)}")
        traceback.print_exc()
        return None

def analyze_collocates(text_dirs, output_dir, word_of_interest, start_year, end_year, collocate_window, window_size=10):
    def read_texts_by_year(directories):
        texts_by_year = defaultdict(list)
        file_paths = [os.path.join(dir, f) for dir in directories for f in os.listdir(dir) if f.endswith('.txt')]
        
        print("Reading files...")
        with concurrent.futures.ThreadPoolExecutor() as executor:
            results = list(tqdm(executor.map(process_file, file_paths), total=len(file_paths)))
        
        for result in results:
            if result:
                year, text = result
                if start_year <= year <= end_year:
                    texts_by_year[year].append(text)
        
        return texts_by_year

    print(f"Starting text processing for years {start_year}-{end_year}...")
    texts_by_year = read_texts_by_year(text_dirs)
    file_directory_map = build_file_directory_map(text_dirs)
    
    all_years = sorted(texts_by_year.keys())
    if not all_years:
        print(f"No texts found in the specified year range {start_year}-{end_year}")
        return
        
    window_data = [
        (window_start, window_start + window_size - 1, 
         [text for year in range(window_start, window_start + window_size) for text in texts_by_year.get(year, [])])
        for window_start in range(start_year, end_year - window_size + 2)
    ]

    collocates_by_window = {}
    print("Processing windows...")
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(lambda data: process_window(data, word_of_interest, collocate_window, file_directory_map), window_data), total=len(window_data)))

    for result in results:
        if result:
            window, window_result = result
            collocates_by_window[window] = window_result

    os.makedirs(output_dir, exist_ok=True)
    print("Saving results to CSV and TXT files...")

    for window, data in collocates_by_window.items():
        save_collocates_to_csv(output_dir, window, data)
        save_contexts_to_csv(output_dir, window, data)
        save_detailed_report(output_dir, window, data, word_of_interest)

    print(f"Collocate analysis completed. Results saved in the '{output_dir}' directory.")


#data saving functions --------------------------------------------   
import csv
import os
import re

def save_collocates_to_csv(output_dir, window, data):

    csv_filename = os.path.join(output_dir, f'collocates_{window}.csv')
    os.makedirs(os.path.dirname(csv_filename), exist_ok=True)
    
    try:
        with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['No.', 'Word', 'Total no. in window corpus', 'Expected collocate frequency',
                             'Observed collocate frequency', 'In no. of texts', 'Mutual Information value'])
           
            for i, collocate in enumerate(data['collocates'], 1):
                writer.writerow([
                    i,
                    collocate['word'],
                    collocate['total_corpus'],
                    f"{collocate['expected_freq']:.2f}",
                    collocate['observed_freq'],
                    collocate['num_texts'],
                    f"{collocate['pmi']:.2f}"
                ])
        print(f"Saved collocates to {csv_filename}")
    except Exception as e:
        print(f"Error saving collocates to {csv_filename}: {e}")

def save_contexts_to_csv(output_dir, window, data):
    
    subfolder_path = os.path.join(output_dir, 'contexts')
    os.makedirs(subfolder_path, exist_ok=True)
    csv_filename = os.path.join(subfolder_path, f'contexts_{window}.csv')
    
    try:
        with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['Context', 'Collocate', 'File', 'Directory', 'Year', 'Frequency Filter'])
            
            for context_info, other_word, filename, dir_name, freq_filter in data['contexts']:
                match = re.search(r'_(\d{4})_', filename)
                year = match.group(1) if match else 'Unknown'
                writer.writerow([context_info, other_word, filename, dir_name, year, freq_filter])
        
        print(f"Saved contexts to {csv_filename}")
    except Exception as e:
        print(f"Error saving contexts to {csv_filename}: {e}")

def save_detailed_report(output_dir, window, data, word_of_interest):
    
    subfolder_path = os.path.join(output_dir, 'collocation_txt')
    os.makedirs(subfolder_path, exist_ok=True)
    txt_filename = os.path.join(subfolder_path, f'collocates_txt_{window}.txt')
    
    try:
        with open(txt_filename, 'w', encoding='utf-8') as txtfile:
            txtfile.write("https://example.com/collocation-analysis\n")
            txtfile.write(f"There are {data['total_tokens']} different words in your collocation database for \"{word_of_interest}\". ")
            txtfile.write(f"(Your query returned {sum(c['observed_freq'] for c in data['collocates'])} matches in {data['total_texts']} different texts)\n")
            txtfile.write("__________________\n\n")
           
            txtfile.write("No.\tWord\tTotal no. in whole corpus\tExpected collocate frequency\t")
            txtfile.write("Observed collocate frequency\tIn no. of texts\tMutual information value\n\n")
           
            for i, collocate in enumerate(data['collocates'], 1):
                txtfile.write(f"{i}\t{collocate['word']}\t{collocate['total_corpus']}\t")
                txtfile.write(f"{collocate['expected_freq']:.2f}\t{collocate['observed_freq']}\t")
                txtfile.write(f"{collocate['num_texts']}\t{collocate['pmi']:.2f}\n")
        
        print(f"Saved detailed report to {txt_filename}")
    except Exception as e:
        print(f"Error saving detailed report to {txt_filename}: {e}")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Igiba\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import concurrent.futures

tasks = [
    #([r"D:\Fact_fiction_corpus\texts\royal society\txt_rstb", 
     # r"D:\Fact_fiction_corpus\texts\royal society\txt_rstl", 
     # r"D:\Fact_fiction_corpus\texts\royal society\txt_rsta"], 
     #"collocation_results/FACT/collocate_results_1665-1958_FACT_css3_w3", "fact", 1665, 1958, 3), #collocate_window = 3/5
    
    #([r"D:\Fact_fiction_corpus\texts\royal society\txt_rstb", 
     # r"D:\Fact_fiction_corpus\texts\royal society\txt_rstl", 
      #r"D:\Fact_fiction_corpus\texts\royal society\txt_rsta"], 
     #"collocation_results/FACTS/collocate_results_1665-1958_FACTS_css3_w3", "facts", 1665, 1958, 3), #scaling is more unforgiving for facts (imperfect) BUT SETTING IT BELOW 3 IS TOO FORGIVING
    
    ([r"D:\Fact_fiction_corpus\texts\royal society\txt_rstb", 
      r"D:\Fact_fiction_corpus\texts\royal society\txt_rstl", 
     r"D:\Fact_fiction_corpus\texts\royal society\txt_rsta"], 
    "collocation_results/FACT2/collocate_results_1665-1958_FACT_css3_w5", "fact", 1665, 1958, 5),

    ([r"D:\Fact_fiction_corpus\texts\royal society\txt_rstb", 
      r"D:\Fact_fiction_corpus\texts\royal society\txt_rstl", 
      r"D:\Fact_fiction_corpus\texts\royal society\txt_rsta"], 
     "collocation_results/FACTS2/collocate_results_1665-1958_FACTS_css3_w5", "facts", 1665, 1958, 5) 
]

def run_analyze_collocates(args):
    analyze_collocates(*args)

with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(run_analyze_collocates, task) for task in tasks]
    for future in concurrent.futures.as_completed(futures):
        try:
            future.result()
        except Exception as e:
            print(f"Error occurred: {e}")

Starting text processing for years 1665-1958...
Starting text processing for years 1665-1958...
Reading files...
Reading files...


  0%|          | 0/18194 [00:00<?, ?it/s]

  0%|          | 0/18194 [00:00<?, ?it/s]

Processing windows...
Processing windows...


  0%|          | 0/285 [00:00<?, ?it/s]

  0%|          | 0/285 [00:00<?, ?it/s]

Window 1673-1682: Using frequency filter of 4 for 514033 total words
Window 1678-1687: Using frequency filter of 4 for 523497 total wordsWindow 1673-1682: Using frequency filter of 4 for 514033 total words

Window 1674-1683: Using frequency filter of 4 for 544332 total words
Window 1676-1685: Using frequency filter of 4 for 572826 total words
Window 1671-1680: Using frequency filter of 5 for 732750 total words
Window 1672-1681: Using frequency filter of 5 for 618805 total words
Window 1672-1681: Using frequency filter of 5 for 618805 total words
Window 1678-1687: Using frequency filter of 4 for 523497 total words
Window 1679-1688: Using frequency filter of 4 for 523497 total words
Window 1671-1680: Using frequency filter of 5 for 732750 total words
Window 1677-1686: Using frequency filter of 5 for 624939 total words
Window 1677-1686: Using frequency filter of 5 for 624939 total words
Window 1679-1688: Using frequency filter of 4 for 523497 total words
Window 1680-1689: Using frequency 

---
extra stuff
---

ungodly messy, could use some clean up

In [3]:
#so my pc turns off eventually
import os
import time

time.sleep(300)
os.system("shutdown /s /t 1")

0

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from collections import defaultdict
import os
import re
from tqdm import tqdm
import string
from nltk.tokenize import word_tokenize

def count_words_in_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
    tokens = word_tokenize(text.lower())
    return len([token for token in tokens if token.isalpha() and token not in string.punctuation])

def count_words_per_year(text_dirs, start_year, end_year):
    def extract_year_from_filename(filename):
        match = re.search(r'rst[bla]?_?(\d{4})', filename)
        return int(match.group(1)) if match else None
    
    word_counts = {
        'rsta': defaultdict(int),
        'rstb': defaultdict(int),
        'rstl': defaultdict(int)
    }
    
    file_paths = []
    for dir in text_dirs:
        for f in os.listdir(dir):
            if f.endswith('.txt'):
                file_paths.append(os.path.join(dir, f))
    
    for file_path in tqdm(file_paths):
        year = extract_year_from_filename(os.path.basename(file_path))
        pub_type = os.path.basename(os.path.dirname(file_path))[-4:]
        
        if year and start_year <= year <= end_year:
            # For RSTA and RSTB, only count years from 1887 onwards
            if (pub_type in ['rsta', 'rstb'] and year >= 1887) or pub_type == 'rstl':
                word_count = count_words_in_file(file_path)
                word_counts[pub_type][year] += word_count
    
    return word_counts

# Directories for Royal Society publications
text_dirs = [
    r"D:\Fact_fiction_corpus\texts\royal society\txt_rstb",
    r"D:\Fact_fiction_corpus\texts\royal society\txt_rstl", 
    r"D:\Fact_fiction_corpus\texts\royal society\txt_rsta"
]

# Count words per year for the historical period
historical_word_counts = count_words_per_year(
    text_dirs,
    start_year=1665,
    end_year=1958
)

# Prepare data for plotting
publication_types = {
    'rsta': {'color': 'red', 'name': 'RSTA', 'order': 3},
    'rstb': {'color': 'blue', 'name': 'RSTB', 'order': 2},
    'rstl': {'color': 'green', 'name': 'RSTL', 'order': 1}
}

# Create the interactive stacked bar plot
fig = go.Figure()

# Generate list of all unique years
all_years = sorted(set().union(
    historical_word_counts['rsta'].keys(),
    historical_word_counts['rstb'].keys(),
    historical_word_counts['rstl'].keys()
))

# Sort publication types by order for proper stacking
sorted_pub_types = sorted(publication_types.items(), key=lambda x: x[1]['order'])

# Add stacked bars for each publication type
for pub_type, details in sorted_pub_types:
    # Filter out zero-value years to reduce noise
    years = [year for year in all_years if historical_word_counts[pub_type].get(year, 0) > 0]
    counts = [historical_word_counts[pub_type].get(year, 0) for year in years]
    
    if years:  # Ensure there are non-zero counts
        first_year = years[0]
        last_year = years[-1]
        
        # Adjust for RSTA and RSTB: ensure it starts from 1887
        if pub_type in ['rsta', 'rstb']:
            first_year = max(first_year, 1887)
        
        fig.add_trace(go.Bar(
            x=years,
            y=counts,
            name=f'{details["name"]} ({first_year}-{last_year})',
            marker_color=details['color'],
            hovertemplate='%{y:,} words<extra></extra>'
        ))

# Update layout to create stacked bars
fig.update_layout(
    title='<b>Total Words per Year by Royal Society Publication Type (1665-1958)<b>',
    xaxis_title='<b>Year<b>',
    yaxis_title='<b>Total Words<b>',
    barmode='stack',  # Properly stack the bars
    template='plotly_white',
    width=1400,  
    height=700,
    font=dict(family='Arial, sans-serif'),
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='center',
        x=0.5,
        font=dict(size=14,  weight='bold')
    ),
    plot_bgcolor='rgba(240,240,240,0.5)',
    margin=dict(t=100, b=50, l=50, r=50)
)

# Customize y-axis with comma-separated thousands
fig.update_yaxes(
    tickformat="~s",
    tickfont=dict(size=12),
    title_font=dict(size=14),
    tickvals=[500_000, 1_000_000, 1_500_000, 2_000_000]  # Set specific tick values
)

fig.update_layout(
    width=1200,  
    height=600,
)

# Show the plot and save HTML
fig.show()
fig.write_html("C:/Users/Igiba/Documents/fact_fiction_project/extra_plots/royal_society_word_counts_annual_bar_plot.html")

In [1]:
import pandas as pd
import plotly.graph_objects as go
import os

def find_highest_numbers_in_directory(directory_path):
    highest_numbers = {}

    for filename in os.listdir(directory_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory_path, filename)
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    highest_number = 0  # Initialize highest number for the current file
                    for line in f.readlines()[1:]:  # Skip the header line
                        parts = line.split('\t')
                        if len(parts) > 0:  # Check if the line is properly formatted
                            try:
                                number = int(parts[0])  # Get the number from the first column
                                highest_number = max(highest_number, number)  # Update highest number
                            except ValueError:
                                pass  # Ignore non-integer values
                    
                    # Store the highest number for the current file
                    highest_numbers[filename] = highest_number

            except PermissionError:
                pass  # Ignore permission errors
            except FileNotFoundError:
                pass  # Ignore file not found errors
            except Exception:
                pass  # Ignore all other exceptions

    return highest_numbers

# Example usage
collocate_directory_path = r"collocate_results_combined_fact_10_window"
highest_numbers = find_highest_numbers_in_directory(collocate_directory_path)

# Prepare the data for plotting
df = pd.DataFrame(list(highest_numbers.items()), columns=['File', 'Highest Number'])

# Modify the 'File' column to extract just the year range
df['Year Range'] = df['File'].str.extract(r'(\d{4}-\d{4})')[0]  # Extract year range using regex
df['Year Range'] = df['Year Range'].fillna(df['File'])  # In case of no match, keep the original filename

# Create the interactive plot
fig = go.Figure()

# Add bars for each file's highest number
fig.add_trace(go.Bar(
    x=df['Year Range'],  # Use the modified Year Range for x-axis
    y=df['Highest Number'],
    marker=dict(color='blue'),
    text=df['Highest Number'],  # Show counts on bars
    textposition='auto',  # Position text on the bars
    hovertemplate='Year Range: %{x}<br>%{y}<extra></extra>'  # Customize hover text
))

# Update layout
fig.update_layout(
    title='Amount of Collocates per Year Window',
    xaxis_title='Year Range',
    yaxis_title='Highest Number',
    xaxis_tickangle=-45,  # Rotate x-axis labels for better visibility
    template='plotly_white'
)

# Show the plot
fig.show()


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'collocate_results_combined_fact_10_window'